# Assignment 2_Part 2
## Author: Lei Cao

In [ ]:
import numpy as np
import pandas as pd
from numpy import arange
from pandas import set_option
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn import model_selection
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

from matplotlib import pyplot

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

### Q3: Pipelines
#### 1. In a new notebook, use the same data pre-processing and prep the data (from Q1)
#### 2. But now, use the Pipeline instead of SpotCheck
#### 3. From the nine possible models listed in Q2, pick the three that worked the best, and let's explore them in more detail
##### 1) Use standard scaling and PCA as pre-processing
##### 2) Do hyperparameter tuning like they do in the blog
#### 4. Use the best fitting model and architecture to predict the holdout data (re-train on all training data, then apply to holdout model)
##### 1) Run accuracy metrics - did your results improve over the previous section? Explain why you think this happened

In [ ]:
# Read in the dataset
df = pd.read_csv('~/500Cities_Data.csv')
print(df.shape)
df.head()

(500, 34)


,Unnamed: 0,StateAbbr,PlaceName,PlaceFIPS,Population2010,ACCESS2_CrudePrev,ARTHRITIS_CrudePrev,BINGE_CrudePrev,BPHIGH_CrudePrev,BPMED_CrudePrev,CANCER_CrudePrev,CASTHMA_CrudePrev,CHD_CrudePrev,CHECKUP_CrudePrev,CHOLSCREEN_CrudePrev,COLON_SCREEN_CrudePrev,COPD_CrudePrev,COREM_CrudePrev,COREW_CrudePrev,CSMOKING_CrudePrev,DENTAL_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,KIDNEY_CrudePrev,LPA_CrudePrev,MAMMOUSE_CrudePrev,MHLTH_CrudePrev,OBESITY_CrudePrev,PAPTEST_CrudePrev,PHLTH_CrudePrev,SLEEP_CrudePrev,STROKE_CrudePrev,TEETHLOST_CrudePrev,Geolocation
0,1,CA,Folsom,624638,72203,7.5,16.9,21.8,25.7,64.8,5.8,8.6,4.1,64.7,78.1,76.6,4.1,37.1,33.3,12.2,74.7,6.7,29.1,2.1,14.3,80.4,9.9,23.8,84.3,8.9,33.9,1.9,6.8,"(38.67504943280, -121.147605753)"
1,2,FL,Largo,1239425,77648,19.6,30.6,16.9,36.1,81.0,9.0,7.9,9.8,77.5,80.2,64.6,10.0,33.7,33.2,20.7,58.6,12.1,39.0,3.7,31.0,75.7,13.1,28.3,77.1,15.4,37.7,4.5,18.3,"(27.90909077340, -82.7714203383)"
2,3,CA,Berkeley,606000,112580,7.7,15.1,19.6,20.9,68.2,4.9,8.8,3.7,64.7,70.0,75.4,3.7,38.2,36.6,11.2,70.0,6.5,27.1,2.1,14.2,81.5,10.9,18.5,83.2,8.2,32.2,1.9,6.7,"(37.87256787650, -122.274907975)"
3,4,CA,Napa,650258,76915,12.3,20.7,19.2,28.1,70.2,6.5,8.9,5.8,63.8,75.4,69.3,5.9,37.9,30.3,14.5,70.2,8.9,34.1,2.8,19.8,76.7,12.0,24.0,83.9,12.0,32.7,2.8,11.2,"(38.29804246490, -122.301093331)"
4,5,FL,Sunrise,1269700,84439,22.8,22.8,16.3,33.3,76.7,6.5,8.0,6.7,77.7,78.7,59.7,7.0,30.5,26.2,16.5,61.0,12.1,37.1,3.2,29.5,82.5,12.7,28.1,81.3,13.3,38.1,3.7,16.2,"(26.15468783030, -80.2998411020)"


In [ ]:
# Drop any rows with NA
df = df.dropna()
print(df.shape)
df.head()

(453, 34)


,Unnamed: 0,StateAbbr,PlaceName,PlaceFIPS,Population2010,ACCESS2_CrudePrev,ARTHRITIS_CrudePrev,BINGE_CrudePrev,BPHIGH_CrudePrev,BPMED_CrudePrev,CANCER_CrudePrev,CASTHMA_CrudePrev,CHD_CrudePrev,CHECKUP_CrudePrev,CHOLSCREEN_CrudePrev,COLON_SCREEN_CrudePrev,COPD_CrudePrev,COREM_CrudePrev,COREW_CrudePrev,CSMOKING_CrudePrev,DENTAL_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,KIDNEY_CrudePrev,LPA_CrudePrev,MAMMOUSE_CrudePrev,MHLTH_CrudePrev,OBESITY_CrudePrev,PAPTEST_CrudePrev,PHLTH_CrudePrev,SLEEP_CrudePrev,STROKE_CrudePrev,TEETHLOST_CrudePrev,Geolocation
0,1,CA,Folsom,624638,72203,7.5,16.9,21.8,25.7,64.8,5.8,8.6,4.1,64.7,78.1,76.6,4.1,37.1,33.3,12.2,74.7,6.7,29.1,2.1,14.3,80.4,9.9,23.8,84.3,8.9,33.9,1.9,6.8,"(38.67504943280, -121.147605753)"
1,2,FL,Largo,1239425,77648,19.6,30.6,16.9,36.1,81.0,9.0,7.9,9.8,77.5,80.2,64.6,10.0,33.7,33.2,20.7,58.6,12.1,39.0,3.7,31.0,75.7,13.1,28.3,77.1,15.4,37.7,4.5,18.3,"(27.90909077340, -82.7714203383)"
2,3,CA,Berkeley,606000,112580,7.7,15.1,19.6,20.9,68.2,4.9,8.8,3.7,64.7,70.0,75.4,3.7,38.2,36.6,11.2,70.0,6.5,27.1,2.1,14.2,81.5,10.9,18.5,83.2,8.2,32.2,1.9,6.7,"(37.87256787650, -122.274907975)"
3,4,CA,Napa,650258,76915,12.3,20.7,19.2,28.1,70.2,6.5,8.9,5.8,63.8,75.4,69.3,5.9,37.9,30.3,14.5,70.2,8.9,34.1,2.8,19.8,76.7,12.0,24.0,83.9,12.0,32.7,2.8,11.2,"(38.29804246490, -122.301093331)"
4,5,FL,Sunrise,1269700,84439,22.8,22.8,16.3,33.3,76.7,6.5,8.0,6.7,77.7,78.7,59.7,7.0,30.5,26.2,16.5,61.0,12.1,37.1,3.2,29.5,82.5,12.7,28.1,81.3,13.3,38.1,3.7,16.2,"(26.15468783030, -80.2998411020)"


In [ ]:
# Recode the target variable
df['Flag_Population'] = (df['Population2010'] > np.median(df['Population2010'])) * 1
print(df.shape)
print(df.groupby('Flag_Population').size())

(453, 35)
Flag_Population
0    227
1    226
dtype: int64


In [ ]:
# Prepare data
array = df.values
X = array[:, 5:33]
Y = array[:, 34]

In [ ]:
X = X.astype(float)
Y = Y.astype(int)

In [ ]:
# Split-out validation dataset
validation_size = 0.20
seed = 123
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size = validation_size, random_state = seed)

In [ ]:
# Based on the results of the nine models from Q2, the LinearDiscriminantAnalysis, LogisticRegression, and ExtraTreesClassifier are three models that work the best

# Construct pipelines using standard scaling and PCA
## LinearDiscriminantAnalysis()
pipe_lda = Pipeline([('scl', StandardScaler()), ('pca', PCA(n_components=2)), ('clf', LinearDiscriminantAnalysis())])

## LogisticRegression()
pipe_lr = Pipeline([('scl', StandardScaler()), ('pca', PCA(n_components=2)), ('clf', LogisticRegression(max_iter=1000000))])

## ExtraTreesClassifier()
pipe_et = Pipeline([('scl', StandardScaler()), ('pca', PCA(n_components=2)), ('clf', ExtraTreesClassifier(random_state=seed))])

In [ ]:
# Set grid search parameters
# Because Linear Discriminant Analysis has no hyperparameters to tune, from now steps are done to solely the Logistic Regression and Extra Trees Classifier models
param_range = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
param_range_fl = [1.0, 0.5, 0.1]

grid_params_lr = [{'clf__penalty': ['l1', 'l2'], 'clf__C': param_range_fl, 'clf__solver': ['liblinear']}]

grid_params_et = [{'clf__criterion': ['gini', 'entropy'], 'clf__min_samples_leaf': param_range, 'clf__max_depth': param_range, 'clf__min_samples_split': param_range[1:]}]

In [ ]:
# Construct grid searches
jobs = -1

## LogisticRegression()
gs_lr = GridSearchCV(estimator = pipe_lr, param_grid = grid_params_lr, scoring = 'accuracy', cv=10)

## ExtraTreesClassifier()
gs_et = GridSearchCV(estimator = pipe_et, param_grid = grid_params_et, scoring = 'accuracy', cv = 10, n_jobs = jobs)

In [ ]:
# List of pipelines for ease of iteration
grids = [gs_lr, gs_et]

# Dictionary of pipelines and classifier types for ease of reference
grid_dict = {0: 'Logistic Regression', 1: 'Extra Trees Classifier'}

In [ ]:
# Fit the grid search objects
print('Performing model optimizations...')
best_acc = 0.0
best_clf = 0
best_gs = ''

for idx, gs in enumerate(grids):
	print('\nEstimator: %s' % grid_dict[idx])	
	# Fit grid search	
	gs.fit(X_train, Y_train)
	# Best params
	print('Best params: %s' % gs.best_params_)
	# Best training data accuracy
	print('Best training accuracy: %.3f' % gs.best_score_)
	# Predict on test data with best params
	Y_pred = gs.predict(X_validation)
	# Test data accuracy of model with best params
	print('Test set accuracy score for best params: %.3f ' % accuracy_score(Y_validation, Y_pred))
	# Track best (highest test accuracy) model
	if accuracy_score(Y_validation, Y_pred) > best_acc:
		best_acc = accuracy_score(Y_validation, Y_pred)
		best_gs = gs
		best_clf = idx
print('\nClassifier with best test set accuracy: %s' % grid_dict[best_clf])

Performing model optimizations...

Estimator: Logistic Regression
Best params: {'clf__C': 0.1, 'clf__penalty': 'l1', 'clf__solver': 'liblinear'}
Best training accuracy: 0.577
Test set accuracy score for best params: 0.538 

Estimator: Extra Trees Classifier
Best params: {'clf__criterion': 'gini', 'clf__max_depth': 3, 'clf__min_samples_leaf': 2, 'clf__min_samples_split': 10}
Best training accuracy: 0.608
Test set accuracy score for best params: 0.538 

Classifier with best test set accuracy: Logistic Regression


In [ ]:
# By comparing the accuracy of the Logistic Regression model, which has best test set accuracy, and that of the Linear Discriminant Analysis, 
# the Logistic Regression model is the best fitting one

# Make predictions on validation dataset
Model1 = LogisticRegression()
Model1.fit(X_train, Y_train)
predictions = Model1.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.5714285714285714
[[20 24]
 [15 32]]
              precision    recall  f1-score   support

           0       0.57      0.45      0.51        44
           1       0.57      0.68      0.62        47

    accuracy                           0.57        91
   macro avg       0.57      0.57      0.56        91
weighted avg       0.57      0.57      0.57        91



/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [ ]:
# As shown in the accuracy metrics below, my result improved over the previous section. Because when performing cross-validation, 
# there will be a data leakage so GridsearchCV fails to find the best model but just focuses on the cross-validation score. 
# Therefore, using pipeline can solve this problem through model tuning.